In [1]:
import getpass
import json
import math
import os
import psycopg2
import pandas as pd
import time

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from configobj import ConfigObj
from multiprocessing import Pool, RLock
from tqdm import tqdm
from typing import Tuple

from projects.data_cleaning import *
from projects.common import *


In [2]:
db_dir = os.path.abspath('') + "/../../../db"

(query_schema_core,
 query_schema_hosp,
 query_schema_icu,
 query_schema_derived,
 conn) = connect_to_database(db_dir)


Database: mimiciv
Username: mimiciv
>>>>> Connected to DB <<<<<


In [9]:
icustays_df = get_database_table_as_dataframe(conn, query_schema_icu, 'icustays')

stay_id = icustays_df[['stay_id']].to_numpy()
print(f"\nNumber of entries based on stay_id : {len(stay_id)}")

careunits = icustays_df[['first_careunit', 'last_careunit']].to_numpy()
print(f"\nCare unit types :")
for i in np.unique(careunits).tolist():
    print(i)
print()


Getting icustays data
Number of entries for icustays : 76540
Column names : ['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los']

Number of entries based on stay_id : 76540

Care unit types :
Cardiac Vascular Intensive Care Unit (CVICU)
Coronary Care Unit (CCU)
Medical Intensive Care Unit (MICU)
Medical/Surgical Intensive Care Unit (MICU/SICU)
Neuro Intermediate
Neuro Stepdown
Neuro Surgical Intensive Care Unit (Neuro SICU)
Surgical Intensive Care Unit (SICU)
Trauma SICU (TSICU)



In [12]:
data_mapping = pd.read_csv("../../../"+DATA_ITEM_PATH, sep='\t', header=0)
data_mapping.fillna('None')

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,224385,Intubation,Intubation,procedureevents,1-Intubation/Extubation,None,Processes,None,None
1,225448,Percutaneous Tracheostomy,Percutaneous Tracheostomy,procedureevents,1-Intubation/Extubation,None,Processes,None,None
2,225468,Unplanned Extubation (patient-initiated),Unplanned Extubation (patient-initiated),procedureevents,1-Intubation/Extubation,None,Processes,None,None
3,225477,Unplanned Extubation (non-patient initiated),Unplanned Extubation (non-patient initiated),procedureevents,1-Intubation/Extubation,None,Processes,None,None
4,226237,Open Tracheostomy,Open Tracheostomy,procedureevents,1-Intubation/Extubation,None,Processes,None,None
...,...,...,...,...,...,...,...,...,...
3856,223877,ZCath Lab Intake,ZCath Lab Intake,chartevents,ZIntake,mL,Numeric,None,None
3857,223878,ZOR Intake,ZOR Intake,chartevents,ZIntake,mL,Numeric,None,None
3858,223879,ZPACU Intake,ZPACU Intake,chartevents,ZIntake,mL,Numeric,None,None
3859,223880,ZFree Form Intake,ZFree Form Intake,chartevents,ZIntake,mL,Numeric,None,None
